In [214]:
# ! jupyter nbconvert --to python --RegexRemovePreprocessor.patterns="^%"  train_and_run.ipynb

# RULES:
# 1. Do not make plot.show() only plot.savefig()

In [2]:
%load_ext autoreload
%autoreload 2
import socket
print(socket.gethostname())

DLBox-Alpha


In [68]:
from fande.utils.find_atomic_groups import find_atomic_groups

from ase import io

data_folder = "/data1/simulations/"

# traj = traj_300 = io.read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = ":")
traj_2000 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/2023-10-19_15%3A10%3A59.497414/md_trajectory.traj",index = "0:10")


sample_snapshot = traj_2000[0]

atomic_groups = find_atomic_groups(sample_snapshot)

from ase.visualize import view

tr = []
for ag in atomic_groups:
        tr.append(sample_snapshot[ag])

view(tr)

Total number of found groups: 14
Checking if all atoms are covered:  True


<Popen: returncode: None args: ['/home/dlbox2/anaconda3/envs/fande/bin/pytho...>

## Make sweep

In [35]:
# # https://docs.wandb.ai/guides/sweeps/walkthrough

import os
machine_name = os.uname()[1]
import wandb
wandb.init(project="rotor-gp", save_code=True, notes="hello", id=machine_name, mode='disabled')

# sweep_configuration = {
#     'method': 'random',
#     'metric': 
#     {
#         'goal': 'minimize', 
#         'name': 'score'
#         },
#     'parameters': 
#     {
#         'x': {'max': 0.1, 'min': 0.01},
#         'y': {'values': [1, 3, 7]},
#      }
# }

## Data loading part

In [6]:
from prepare_model import prepare_data, sample_data, prepare_model


In [28]:
# Hyperparameters:
hparams = {
        'dtype' : 'float32',
        'device' : 'gpu'
        }

# Descriptors parameters:
# https://github.com/lab-cosmo/librascal/blob/master/examples/MLIP_example.ipynb
soap_params = {
# 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
'interaction_cutoff': 3.5,
'gaussian_sigma_constant': 0.3,
'max_radial': 4,
'max_angular': 4,
'cutoff_smooth_width': 0.1,
# 'average': "off",
# 'crossover': True,
# 'dtype': "float64",
# 'n_jobs': 10,
# 'sparse': False,
# 'positions': [7, 11, 15] # ignored
}

fdm = prepare_data(hparams, soap_params, traj_sample_rate=1)


Saving data to directory:  /home/dlbox2/repos/rotor-gp/results/train_and_run/2023-11-10_11 46-48_336708
Loading training data...
Length of the train trajectory:  10
Total number of found groups: 14
Checking if all atoms are covered:  True
Total number of frames is 10
Total number of batches is 10
Total length of traj is 10
Total number of batches 10
Calculating invariants on trajectory with librascal...


100%|██████████| 10/10 [00:05<00:00,  1.91it/s]


Total number of frames is 20
Total number of batches is 20
Total length of traj is 20
Total number of batches 20
Calculating invariants on trajectory with librascal...


100%|██████████| 20/20 [00:10<00:00,  1.95it/s]



-----------
Group  0
-----------
Number of training points for group 0:  120
Number of test points for group 0:  240

-----------
Group  1
-----------
Number of training points for group 1:  720
Number of test points for group 1:  1440

-----------
Group  2
-----------
Number of training points for group 2:  360
Number of test points for group 2:  720

-----------
Group  3
-----------
Number of training points for group 3:  180
Number of test points for group 3:  360

-----------
Group  4
-----------
Number of training points for group 4:  900
Number of test points for group 4:  1800

-----------
Group  5
-----------
Number of training points for group 5:  420
Number of test points for group 5:  840

-----------
Group  6
-----------
Number of training points for group 6:  180
Number of test points for group 6:  360

-----------
Group  7
-----------
Number of training points for group 7:  180
Number of test points for group 7:  360

-----------
Group  8
-----------
Number of training p

In [26]:
#[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 

train_data_loaders = sample_data(fdm, N_samples=6000)

Taking ALL 120 samples for group 0
Dataloader for group 0 created
Number of samples in dataloader: 120
Taking ALL 720 samples for group 1
Dataloader for group 1 created
Number of samples in dataloader: 720
Taking ALL 360 samples for group 2
Dataloader for group 2 created
Number of samples in dataloader: 360
Taking ALL 180 samples for group 3
Dataloader for group 3 created
Number of samples in dataloader: 180
Taking ALL 900 samples for group 4
Dataloader for group 4 created
Number of samples in dataloader: 900
Taking ALL 420 samples for group 5
Dataloader for group 5 created
Number of samples in dataloader: 420
Taking ALL 180 samples for group 6
Dataloader for group 6 created
Number of samples in dataloader: 180
Taking ALL 180 samples for group 7
Dataloader for group 7 created
Number of samples in dataloader: 180
Taking ALL 180 samples for group 8
Dataloader for group 8 created
Number of samples in dataloader: 180
Taking ALL 120 samples for group 9
Dataloader for group 9 created
Number 

## Training part

In [30]:
import os
os.getcwd()

'/home/dlbox2/repos/rotor-gp/results/train_and_run/2023-11-10_11 46-48_336708'

In [36]:
hparams['dtype'] = 'float32'
hparams['device'] = 'gpu'

# per AG steps and learning rate
# n_steps = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 250] 
n_steps = 250
learning_rate = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

AG_force_model = prepare_model(
    fdm,
    train_data_loaders, 
    hparams, soap_params, 
    n_steps, 
    learning_rate, 
    gpu_id=0)


### TESTING PREDICITONS ###

# os.makedirs("")

from fande.predict import PredictorASE


AG_force_model.eval()

predictor = PredictorASE(
            fdm,
            AG_force_model,
            hparams,
            soap_params
)

rmse_per_model, mae_per_model = predictor.test_errors(view_worst_atoms=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized

ModelForces initialized


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU avai

Training force model 0 (Total 14 models)


Training: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=250` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type                       | Params
----------------------------------------------------------
0 | likelihood | GaussianLikelihood         | 1     
1 | model      | ExactGPModelForces         | 1.2 K 
2 | mll        | ExactMarginalLogLikelihood | 1.2 K 
----------------------------------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model params size (MB)


Training force model 13 (Total 14 models)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=250` reached.


objects of fande.predict module imported...
Error metrics for atomic group  0
RMSE:  0.79309404
MAE:  0.63628894
Max error:  1.6217387
Error metrics for atomic group  1
RMSE:  0.1670089
MAE:  0.12725107
Max error:  0.6506858
Error metrics for atomic group  2
RMSE:  0.30333412
MAE:  0.24041076
Max error:  1.11738
Error metrics for atomic group  3
RMSE:  0.3705283
MAE:  0.26316267
Max error:  1.2913754
Error metrics for atomic group  4
RMSE:  0.52485675
MAE:  0.38730466
Max error:  2.0218742
Error metrics for atomic group  5
RMSE:  0.5181346
MAE:  0.39270958
Max error:  1.5695975
Error metrics for atomic group  6
RMSE:  0.850116
MAE:  0.6681914
Max error:  2.2136555
Error metrics for atomic group  7
RMSE:  0.6203805
MAE:  0.49547115
Max error:  1.5787879
Error metrics for atomic group  8
RMSE:  0.5492843
MAE:  0.45603433
Max error:  1.1012394
Error metrics for atomic group  9
RMSE:  0.40442204
MAE:  0.34821278
Max error:  0.8446031
Error metrics for atomic group  10
RMSE:  0.39935058
MAE

In [37]:
print(*rmse_per_model)

0.79309404 0.1670089 0.30333412 0.3705283 0.52485675 0.5181346 0.850116 0.6203805 0.5492843 0.40442204 0.39935058 0.8778609 0.5266907 0.15110892


In [38]:
print(*rmse_per_model)

0.79309404 0.1670089 0.30333412 0.3705283 0.52485675 0.5181346 0.850116 0.6203805 0.5492843 0.40442204 0.39935058 0.8778609 0.5266907 0.15110892


## Testing part

In [39]:
### TESTING PREDICITONS ###

# os.makedirs("")

from fande.predict import PredictorASE


AG_force_model.eval()

predictor = PredictorASE(
            fdm,
            AG_force_model,
            hparams,
            soap_params
)

rmse_per_model, mae_per_model = predictor.test_errors(view_worst_atoms=True)

# rmse_per_model

Error metrics for atomic group  0
RMSE:  0.79309404
MAE:  0.63628894
Max error:  1.6217387
Error metrics for atomic group  1
RMSE:  0.1670089
MAE:  0.12725107
Max error:  0.6506858
Error metrics for atomic group  2
RMSE:  0.30333412
MAE:  0.24041076
Max error:  1.11738
Error metrics for atomic group  3
RMSE:  0.3705283
MAE:  0.26316267
Max error:  1.2913754
Error metrics for atomic group  4
RMSE:  0.52485675
MAE:  0.38730466
Max error:  2.0218742
Error metrics for atomic group  5
RMSE:  0.5181346
MAE:  0.39270958
Max error:  1.5695975
Error metrics for atomic group  6
RMSE:  0.850116
MAE:  0.6681914
Max error:  2.2136555
Error metrics for atomic group  7
RMSE:  0.6203805
MAE:  0.49547115
Max error:  1.5787879
Error metrics for atomic group  8
RMSE:  0.5492843
MAE:  0.45603433
Max error:  1.1012394
Error metrics for atomic group  9
RMSE:  0.40442204
MAE:  0.34821278
Max error:  0.8446031
Error metrics for atomic group  10
RMSE:  0.39935058
MAE:  0.31473038
Max error:  1.3891901
Error me

In [70]:
from prepare_model import prepare_fande_ase_calc

# Hyperparameters:
hparams = {
        'dtype' : 'float32',
        'device' : 'gpu'
        }

# Descriptors parameters:
# https://github.com/lab-cosmo/librascal/blob/master/examples/MLIP_example.ipynb
soap_params = {
# 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
'interaction_cutoff': 3.0,
'gaussian_sigma_constant': 0.3,
'max_radial': 4,
'max_angular': 4,
'cutoff_smooth_width': 0.1,
# 'average': "off",
# 'crossover': True,
# 'dtype': "float64",
# 'n_jobs': 10,
# 'sparse': False,
# 'positions': [7, 11, 15] # ignored
}


fande_calc = prepare_fande_ase_calc(hparams, soap_params)

Saving data to directory:  /home/dlbox2/repos/rotor-gp/results/train_and_run/2023-11-10_12 52-20_488153
Loading training data...


In [58]:
traj_2000 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/2023-10-19_15%3A10%3A59.497414/md_trajectory.traj",index = ":")


In [65]:
len(traj_2000[0:5000:50])

100

In [136]:
# from fande.predict import PredictorASE

# from fande.ase import FandeCalc



# predictor = PredictorASE(
#             fdm,
#             model_e,
#             trainer_e,
#             AG_force_model,
#             # trainer_f,
#             hparams,
#             soap_params
# )

# atoms = traj_test[14].copy()

# # predictor.predict_forces_single_snapshot_r(atoms)

# atoms = traj_test[5].copy()

# atoms.calc = FandeCalc(predictor)

# atoms.get_forces()

In [ ]:
# from matplotlib import pyplot as plt



# traj_1 = traj_300

# traj_2 = traj_1500

# mol = traj_300[200].copy()



# uncertainties_0 = []
# traj_0 = []
# for i in range(0,50):
#     traj_0.append(mol.copy())
#     uncertainties_0.append(FandeCalc(predictor).get_forces_variance(mol.copy())[13,1])
#     mol.positions[13,1] = mol.positions[13,1] + 0.1

# uncertainties_1 = []
# for mol in traj_1[300:320]:
#     uncertainties_1.append(FandeCalc(predictor).get_forces_variance(mol.copy())[212,0])

# uncertainties_2 = []
# for mol in traj_2[300:320]:
#     uncertainties_2.append(FandeCalc(predictor).get_forces_variance(mol.copy())[212,0])

In [ ]:
atoms.get_forces_variance()

In [223]:
# from ase import Atoms
# class AtomsWrapped(Atoms):   
#     def __init__(self, *args, **kwargs):
#         super(AtomsWrapped, self).__init__(*args, **kwargs)      
#         self.calc_history_counter = 0
    
#     def get_forces_variance(self):
#         forces_variance = super(AtomsWrapped, self).calc.get_forces_variance(self)
#         return forces_variance


#     def get_forces(self):       
#         forces = super(AtomsWrapped, self).get_forces()
#         forces_variance = super(AtomsWrapped, self).calc.get_forces_variance(self)
#         self.arrays['forces_variance'] = forces_variance
#         # energy = super(AtomsWrapped, self).get_potential_energy()
#         from ase.io import write
#         import os
#         os.makedirs("ase_calc_history" , exist_ok=True)
#         write( "ase_calc_history/" + str(self.calc_history_counter) + ".xyz", self, format="extxyz")
#         # self.calc_history.append(self.copy())       
#         self.calc_history_counter += 1
#         return forces


from fande.ase import FandeAtomsWrapper 
    
mol = io.read("/home/qklmn/data/starting_configuration/1.cif")

mol = FandeAtomsWrapper(mol)
mol.calc = FandeCalc(predictor)

In [226]:
os.getcwd()

'/misc/home/qklmn'

In [225]:
mol.get_forces_variance()

/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.99it/s]
/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



array([[0.28246722, 0.2569364 , 0.17697105],
       [0.16083393, 0.22986922, 0.22655949],
       [0.26595339, 0.19469914, 0.15795645],
       [0.21528253, 0.12407764, 0.21884617],
       [0.59140593, 0.39451155, 0.44077215],
       [0.43508896, 0.30540356, 0.42349371],
       [0.74045306, 0.78981966, 0.77597141],
       [0.73075187, 0.74705845, 0.75709736],
       [0.74937946, 0.74908561, 0.77944779],
       [0.75806224, 0.73596281, 0.72551358],
       [0.73399311, 0.75456107, 0.74840587],
       [0.71749824, 0.728517  , 0.7514413 ],
       [0.73656768, 0.74442554, 0.72919828],
       [0.73354727, 0.75287962, 0.7241143 ],
       [0.73433924, 0.74929017, 0.83233702],
       [0.76254344, 0.75875998, 0.77559745],
       [0.73451585, 0.77248311, 0.79197532],
       [0.74167603, 0.77837813, 0.76843727],
       [0.70716739, 0.72476864, 0.71955955],
       [0.7083602 , 0.72318393, 0.71659815],
       [0.74912465, 0.76287735, 0.77348232],
       [0.73211652, 0.74619669, 0.76579893],
       [0.

In [165]:
mol.get_forces_variance()

array([[0.28246722, 0.2569364 , 0.17697105],
       [0.16083393, 0.22986922, 0.22655949],
       [0.26595339, 0.19469914, 0.15795645],
       [0.21528253, 0.12407764, 0.21884617],
       [0.59140593, 0.39451155, 0.44077215],
       [0.43508896, 0.30540356, 0.42349371],
       [0.74045306, 0.78981966, 0.77597141],
       [0.73075187, 0.74705845, 0.75709736],
       [0.74937946, 0.74908561, 0.77944779],
       [0.75806224, 0.73596281, 0.72551358],
       [0.73399311, 0.75456107, 0.74840587],
       [0.71749824, 0.728517  , 0.7514413 ],
       [0.73656768, 0.74442554, 0.72919828],
       [0.73354727, 0.75287962, 0.7241143 ],
       [0.73433924, 0.74929017, 0.83233702],
       [0.76254344, 0.75875998, 0.77559745],
       [0.73451585, 0.77248311, 0.79197532],
       [0.74167603, 0.77837813, 0.76843727],
       [0.70716739, 0.72476864, 0.71955955],
       [0.7083602 , 0.72318393, 0.71659815],
       [0.74912465, 0.76287735, 0.77348232],
       [0.73211652, 0.74619669, 0.76579893],
       [0.

In [153]:
mol.calc.get_forces_variance(mol)

/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  2.68it/s]
/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



array([[0.28246722, 0.2569364 , 0.17697105],
       [0.16083393, 0.22986922, 0.22655949],
       [0.26595339, 0.19469914, 0.15795645],
       [0.21528253, 0.12407764, 0.21884617],
       [0.59140593, 0.39451155, 0.44077215],
       [0.43508896, 0.30540356, 0.42349371],
       [0.74045306, 0.78981966, 0.77597141],
       [0.73075187, 0.74705845, 0.75709736],
       [0.74937946, 0.74908561, 0.77944779],
       [0.75806224, 0.73596281, 0.72551358],
       [0.73399311, 0.75456107, 0.74840587],
       [0.71749824, 0.728517  , 0.7514413 ],
       [0.73656768, 0.74442554, 0.72919828],
       [0.73354727, 0.75287962, 0.7241143 ],
       [0.73433924, 0.74929017, 0.83233702],
       [0.76254344, 0.75875998, 0.77559745],
       [0.73451585, 0.77248311, 0.79197532],
       [0.74167603, 0.77837813, 0.76843727],
       [0.70716739, 0.72476864, 0.71955955],
       [0.7083602 , 0.72318393, 0.71659815],
       [0.74912465, 0.76287735, 0.77348232],
       [0.73211652, 0.74619669, 0.76579893],
       [0.

## Compute MD with ML model

In [51]:
### MD with fande calc
print('Testing performance with (meta-)dynamics run...')

from fande.ase import FandeCalc
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal

from ase import io


# from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths
from ase.optimize import BFGS
from ase import units
from ase.io import read
import logging
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen


logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) # logging.ERROR to disable or INFO

data_folder = "/home/qklmn/data/"
traj_test = io.read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = "300:400")

# traj_md = read('../results/test/machine_learning/dftb_opt_1000_six_rings.traj', index=":")
# traj_opt = read('../results/test/machine_learning/opt.traj', index=":")

# atoms = fdm.mol_traj[10].copy()
# atoms = traj_md[300].copy()
# atoms = traj_opt[-1].copy()
atoms = traj_test[10].copy()
atoms.set_pbc(True)


# moving_atoms = sum(atomic_groups, []) 
# fixed_atoms = list( set(range(264)) - set(moving_atoms) )
# fix_atoms = FixAtoms(indices=fixed_atoms)
# atoms.set_constraint(fix_atoms)


# atoms.calc = FandeCalc(predictor)
atoms.calc = fande_calc


# atoms.calc.set_atomic_groups([rings_carbons, rings_hydrogens], titles=["Rings carbons", "Rings hydrogens"])
# atoms.calc.set_forces_errors_plot_file("../results/test/md_runs/forces_errors.png", loginterval=1)
# atoms.calc = LennardJones()

os.makedirs("md_run/", exist_ok=True)

# Verlet dynamics:
# MaxwellBoltzmannDistribution(atoms, temperature_K=300)
# dyn = VelocityVerlet(
#     atoms,
#     dt = 0.1*units.fs,
#     trajectory="md_run/md_test.traj",
#     logfile="md_run/md_log.log",
# )

# dyn = NPT(
#     atoms,
#     # dt = 0.5*units.fs,
#     timestep=0.1,
#     temperature_K=300,
#     externalstress=0.0,
#     trajectory="../results/test/md_runs/md_test.traj",
#     logfile="../results/test/md_runs/md_log.log",
# )

# dyn = NPTBerendsen(atoms, timestep=0.1 * units.fs, temperature_K=300,
#                    taut=100 * units.fs, pressure_au=1.01325 * units.bar,
#                    taup=1000 * units.fs, compressibility=4.57e-5 / units.bar,
#                    trajectory="../results/test/md_runs/md_test.traj",
#                    logfile="../results/test/md_runs/md_log.log",)

# import os

# MaxwellBoltzmannDistribution(atoms, temperature_K=300, force_temp=True)
# dyn = NVTBerendsen(atoms, 0.2 * units.fs, 300, taut=0.5*1000*units.fs, 
#                    trajectory="md_run/md_test.traj",   
#                    logfile="md_run/md_log.log")

# dyn.run(100)

# Langevin dynamics:
# https://databases.fysik.dtu.dk/ase/tutorials/md/md.html
# MaxwellBoltzmannDistribution(atoms, temperature_K=100, force_temp=True)
dyn = Langevin(atoms, 0.2*fs, 
               temperature_K=300, #0.1/units.kB, 
               friction=0.1,
        #        fixcm=True, 
               trajectory='md_run/md_test.traj',
               logfile="md_run/md_log.log")

dyn.run(10)

# # Structure optimization:
# dyn = BFGS(
#     atoms,
#     trajectory="../results/test/md_runs/md_test.traj",
#     logfile="../results/test/md_runs/md_log.log",)
# dyn.run(fmax=0.1)


print(" ALL JOBS WITHIN PYTHON SCRIPT ARE DONE! ")

print("TIMING: ", time.time()-start_time, " seconds")

Testing performance with (meta-)dynamics run...


/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  5.10it/s]
/misc/home/qklmn/repos/rotor-gp/code/../../fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


...

100%|██████████| 1/1 [00:00<00:00,  5.18it/s]


Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  5.20it/s]


 ALL JOBS WITHIN PYTHON SCRIPT ARE DONE! 


NameError: name 'time' is not defined

In [ ]:
# make a nice starting configuration for i-pi
from ase import io

data_folder = "/home/qklmn/data/"
atoms = read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = "350")
# io.write("/home/qklmn/data/starting_configuration/1.cif", atoms, format='cif')

# Make dftb+ trajectory:

In [121]:
from ase import io
# from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths
from ase.optimize import BFGS
from ase import units
from ase.io import read
import logging
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen

from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal



from ase.calculators.dftb import Dftb
import os
os.environ['OMP_NUM_THREADS'] = "6,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /usr/local/dftbplus-21.2/bin/dftb+ > PREFIX.out"
# os.environ["ASE_DFTB_COMMAND"] = "dftb+ > PREFIX.out"
os.environ["DFTB_PREFIX"] = "/home/qklmn/data/dftb/pbc-0-3"




# initial_structure = io.read("/home/qklmn/data/starting_configuration/1.cif")
crystal_264=io.read("/home/qklmn/data/starting_configuration/1.cif")





logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) # logging.ERROR to disable or INFO

data_folder = "/home/qklmn/data/"
traj_test = io.read(data_folder + "datasets/rotors/different_temperatures/300/OUTCAR", format="vasp-out", index = "300:400")

# traj_md = read('../results/test/machine_learning/dftb_opt_1000_six_rings.traj', index=":")
# traj_opt = read('../results/test/machine_learning/opt.traj', index=":")

# atoms = fdm.mol_traj[10].copy()
# atoms = traj_md[300].copy()
# atoms = traj_opt[-1].copy()
# atoms = traj_test[10].copy()

atoms = crystal_264.copy()
atoms.set_pbc(True)


calc = Dftb(atoms=atoms,
        label='crystal',
        # Hamiltonian_ = "xTB",
        # # Hamiltonian_Method = "GFN1-xTB",
        # Hamiltonian_MaxAngularMomentum_='',
        # Hamiltonian_MaxAngularMomentum_O='p',
        # Hamiltonian_MaxAngularMomentum_H='s',
        # Hamiltonian_MaxAngularMomentum_N='s',
        # Hamiltonian_MaxAngularMomentum_C='s',
        # Hamiltonian_MaxAngularMomentum_Si='s',
        kpts=(1,1,1),
        # Hamiltonian_SCC='Yes',
        # Verbosity=0,
        # Hamiltonian_OrbitalResolvedSCC = 'Yes',
        # Hamiltonian_SCCTolerance=1e-15,
        # kpts=None,
        # Driver_='ConjugateGradient',
        # Driver_MaxForceComponent=1e-3,
        # Driver_MaxSteps=200,
        # Driver_LatticeOpt = 'Yes',
        #     Driver_AppendGeometries = 'Yes',
        #     Driver_='',
        #     Driver_Socket_='',
        #     Driver_Socket_File='Hello'
        )

atoms.calc = calc


os.chdir("/home/qklmn/data/datasets/rotors/different_temperatures/dftb")



# Langevin dynamics:
# https://databases.fysik.dtu.dk/ase/tutorials/md/md.html
# MaxwellBoltzmannDistribution(atoms, temperature_K=100, force_temp=True)
dyn = Langevin(atoms, 1*fs, 
               temperature_K=300, #0.1/units.kB, 
               friction=0.1,
        #        fixcm=True, 
               trajectory='traj_300.traj',
               logfile="md_log.log")

dyn.run(1000)

True

In [217]:
import matplotlib.pyplot as plt

traj = io.read("/home/qklmn/data/datasets/rotors/different_temperatures/dftb/traj_2100.traj", index=":")

energies = [mol.get_temperature() for mol in traj]
# plt.plot(energies)
# plt.show()

In [122]:
traj_dftb_2100 = io.read("/home/qklmn/data/datasets/rotors/different_temperatures/dftb/traj_2100.traj", index = ":")

In [123]:
initial_structure = io.read("/home/qklmn/data/starting_configuration/1.cif")

In [218]:
# print(*traj_dftb_2100[0].get_chemical_symbols() )
# print(*initial_structure.get_chemical_symbols() )

False